In [ ]:
#=======================================#
# Yes, this notebook is over-commented. #
#=======================================#

In [1]:
# Make notebook span entire screen, horizontally.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import gym
import numpy as np
import tensorflow as tf

/home/grberlstein/rl_bot/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Code Graveyard Snippits:
#--------------
# indicies = tf.concat(values=[self.col, self.acts], axis=1)
# self.picked_action_prob = tf.gather_nd(self.probabilities,indicies)
# self.losses = -tf.log(tf.clip_by_value(self.picked_action_prob,1e-10,1.0))

In [4]:
# TODO:
#  -> fix loss funtion
#  -> check reward processing

class PolicyAgent(object):
    def __init__(self, sess):
        self.n_actions = 4
        
        # Build graph (i.e. internal neural network).
        self._build()
        
        # Create environment within which the graph will train.
        self.sess = sess
        
        # Evaluate an tf.Operation (returned by global_variables_initializer()) that initializes TRAINABLE VARIABLES.        
    def _build(self):
        self.obs      = tf.placeholder(tf.float32, (None, 8))
        self.acts     = tf.placeholder(tf.int32, (None,1))
        self.target   = tf.placeholder(tf.float32, (None,1))
        self.l_rate   = tf.placeholder(tf.float32)
        self.training = tf.placeholder(tf.bool)
        self.col      = tf.placeholder(tf.int32, (None,1))

        
        flat = tf.layers.flatten(self.obs)
        with tf.variable_scope('actor-hidden'):
            h1 = tf.layers.dense(flat, 128, 
                                 activation=tf.nn.relu, 
                                 kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                 name='h1')
            drop1 = tf.layers.dropout(h1, training=self.training, 
                                      name='drop1')
            h2 = tf.layers.dense(drop1,   64, activation=tf.nn.relu, 
                                 kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                 name='h2')
            drop2 = tf.layers.dropout(h2, training=self.training, 
                                      name='drop2')
            h3 = tf.layers.dense(drop2,   32, activation=tf.nn.relu, 
                                 kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                 name='h3')
            drop3 = tf.layers.dropout(h3, training=self.training, name='dropout')
        
            # This needs to have linear activations to conform to the "unscaled log probabilities" 
            #requirement of `sparse_softmax_cross_entropy_loss_with_logits()`.
            self.out = tf.layers.dense(drop3, self.n_actions, 
                                       kernel_initializer=tf.random_normal_initializer(), 
                                       name='out')
            # Clip values to avoid nans
            
        
        # Compute probabilities associated with each action.
        self.probabilities = tf.clip_by_value(tf.nn.softmax(self.out),1e-10,1.0)
        
        # Compute entropy based on probabilities
        self.entropy = -tf.reduce_sum(self.probabilities * tf.log(tf.clip_by_value(self.probabilities,1e-10,1.0)), 
                                      1, name="entropy")
        
        indicies = tf.concat(values=[self.col, self.acts], axis=1)
        self.picked_action_prob = tf.gather_nd(self.probabilities,indicies)
        self.losses = -(tf.log(tf.clip_by_value(self.picked_action_prob,1e-10,1.0))* self.target + self.entropy*0.01)
        # Compute losses
#         self.neg_log = tf.losses.sparse_softmax_cross_entropy(labels=self.acts, logits=clipped_out)
        self.loss = tf.reduce_sum(self.losses)
        
        # Set optimizer.
        self.train_op = tf.train.AdamOptimizer(self.l_rate).minimize(self.loss)
    
    def choose_action(self, obs, p_flag=False):
        # Compute probabilities associated with each action.
        out_vals, prob_weights = self.sess.run([self.out, self.probabilities], feed_dict={
            self.obs:      np.array(obs).reshape(-1, 8),
            self.training: False
        })
        
        if p_flag: print(prob_weights, out_vals)
        # Choose action based on computed probabilities.
        return np.random.choice(range(prob_weights.shape[1]), p=prob_weights.ravel())
    
    def train(self, act, obs, target, l_rate):
#         print(target)
        length = np.array(act).reshape(-1,1).shape[0]
        _, loss, prob, entropy, out = self.sess.run([self.train_op,self.loss, self.probabilities, self.entropy,
                                                    self.out], feed_dict={
            self.obs:      np.array(obs).reshape(-1, 8),
            self.acts:     np.array(act).reshape(-1,1),
            self.target:   np.array(target).reshape(-1,1),
            self.l_rate:   l_rate,
            self.training: True,
            self.col:      np.arange(length).reshape(-1,1)
        })
#         print('--------------')
#         print(loss)
#         print(prob)
# #         print(entropy)
# #         print(neg_log)
#         print(out)
#         print('--------------')
        return loss


In [5]:
class Critic(object):
    def __init__(self, sess):
        self.sess = sess
        self._build()
        
    def _build(self):
        self.obs     = tf.placeholder(tf.float32, (None, 8))   # The input state
        self.target  = tf.placeholder(tf.float32, (None, 1))             # The target output score
        self.lr      = tf.placeholder(tf.float32)                        # Learning Rate
        
        flat = tf.layers.flatten(self.obs)
        with tf.variable_scope('critic-hidden'):
            h1 = tf.layers.dense(flat, 128, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                 name='h1')
            h2 = tf.layers.dense(h1, 64, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                 name='h2')
            out = tf.layers.dense(h2, 1, kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                 name='out')
            
        self.value_estimate = tf.squeeze(out)   # [[num]] -> num
        self.loss = tf.squared_difference(self.value_estimate, self.target)
        
        self.train_op = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
        
    def predict(self, obs):
        return sess.run(self.value_estimate,feed_dict={self.obs: np.array(obs).reshape(-1, 8)})
    
    def update(self, obs, target, learning_rate=0.01):
        _, loss = self.sess.run([self.train_op, self.loss], feed_dict={self.obs:     np.array(obs).reshape(-1, 8),
                                                                       self.target:  np.array(target).reshape(-1,1),
                                                                       self.lr:      learning_rate})
        return loss
    

In [6]:
class ACHandler(object):
    def __init__(self, actor, critic, sess, env, path='./.model.ckpt'):
        self.actor = actor
        self.critic = critic
        self.env = env
        self.sess = sess
    
        self.saver = tf.train.Saver() # saves all variables
        self.path = path
        
        
        
    def init_vars(self):
        self.sess.run(tf.global_variables_initializer())
        
    def run(self, train_func, rollout=100, a_rate=0.001, c_rate=0.005, decay=0.99, render=False, **kwargs):
        assert isinstance(train_func, str) and train_func.startswith('train_'), \
               'invalid train_func name specified'
        getattr(self, train_func)(self.rollout(rollout, render, decay), a_rate, c_rate, **kwargs)
        
        # Close the display window
        if render: self.env.close()
            
    def run_constant_training(self, num_episodes, a_rate=0.001, c_rate=0.005, decay=0.99, render=False):
        """
        Runs training and updates both networks during every time step
        """
        for _ in range(num_episodes):
            obs_curr = env.reset()
            done = False

            while not done:

                if render: self.env.render()
                action = self.actor.choose_action(obs_curr)

                # Take action in environment.
                next_obs, reward, done, _ = self.env.step(action)

                next_estimate = self.critic.predict(next_obs)
                td_target = reward + decay*next_estimate
                td_error = td_target - self.critic.predict(obs_curr)
                c_loss = self.critic.update(obs_curr, td_target, c_rate)
                a_loss = self.actor.train(action, obs_curr, td_error, a_rate)

                obs_curr = next_obs
                
    def play(self):
        """
        Runs a single instance of the game without training or storing training information
        Always displays the game and closes the window afterward
        """
        obs_curr = self.env.reset()
        done = False
        while not done:
            self.env.render()

            # Agent chooses action based on difference frame.
            action = self.actor.choose_action(obs_curr)

            # Take action in environment.
            next_obs, reward, done, _ = self.env.step(action)
        env.close()
        
    def train_rsample(self, batch, a_rate, c_rate, num_epochs=50, mini_batch_size=100):
        """
        Performs random mini-batch training on both networks from a given
          set of batch information
        """
        for x in range(num_epochs):
            indices = np.random.randint(len(batch['obs']), size=mini_batch_size)
            loss = self.actor.train([batch['act'][i] for i in indices],
                             [batch['obs'][i] for i in indices],
                             [batch['advantage'][i] for i in indices],
                             a_rate)
            self.critic.update([batch['obs'][i] for i in indices],
                               [batch['td_target'][i] for i in indices],
                               c_rate)
#             assert x < 2
            
    def train_all(self, batch, a_rate, c_rate):
        """
        Trains both networks on all peices of inromation in the batch
        """
        self.actor.train(batch['act'],
                         batch['obs'],
                         batch['advantage'],
                         a_rate)
        self.critic.update(batch['obs'],
                           batch['td_target'],
                           c_rate)
    
    def compute_advantage(self, obs, rewards, decay):
        policy_target = np.zeros_like(rewards)
        value_target = np.zeros_like(rewards)
        running_reward = 0
        
#         for idx in reversed(range(len(rewards))):
#             running_reward += rewards[idx] + decay*running_reward
#             policy_target[idx] = running_reward - self.critic.predict(obs[idx])
#             value_target[idx] = running_reward
        for idx in range(len(rewards))[0:-1]:
            next_estimate = self.critic.predict(obs[idx+1])
            td_target = rewards[idx] + decay*next_estimate
            td_error = td_target - self.critic.predict(obs[idx])
            policy_target[idx] = td_error
            value_target[idx] = td_target
        
        return policy_target.tolist(), value_target.tolist()

    def save(self):
        self.saver.save(self.sess, self.path)
        
    def load(self):
        self.saver.restore(self.sess, self.path)
            
    def rollout(self, count, render, decay):
        batch = {'act': [], 'obs': [], 'rew': [], 'advantage':[], 'td_target':[]}
        
        for episode in range(count):
            # Stores all the stuff
            history = {'act': [], 'obs': [], 'rew': [], 'advantage':[], 'td_target':[]}
            
            obs_curr = env.reset()
            done = False
               
            while not done:
                
                if render: self.env.render()
                # Agent chooses action based on difference frame.
                action = self.actor.choose_action(obs_curr, False)
        
                # Take action in environment.
                next_obs, reward, done, _ = self.env.step(action)
                
                history['act'].append(action)
                history['obs'].append(obs_curr)
                history['rew'].append(reward)
                
                obs_curr = next_obs

            # Process rewards per episode.
            history['advantage'], history['td_target'] = self.compute_advantage(history['obs'] + obs_curr, history['rew'], decay)
            # Add episode to batch.
            for key in batch:
                batch[key].extend(history[key])
#             print(sum(history['rew'])) 
        return batch 

In [7]:
tf.reset_default_graph()
env = gym.make('LunarLander-v2') # RGB observation space

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [8]:
sess = tf.Session()
actor = PolicyAgent(sess)
critic = Critic(sess)

In [9]:
handler = ACHandler(actor, critic, sess, env, '.models/l1.cpt')

In [10]:
handler.init_vars()

In [ ]:
while(True):
    for _ in range(100):
        handler.run('train_rsample', rollout=40, a_rate=0.001, c_rate=0.01, decay=0.98, render=False,
                   num_epochs=100, mini_batch_size=100)
        print('-',end='')
    print('\nCompleted 100 Training Iterations\n')
    handler.save()

------------

In [ ]:
while(True):
    for _ in range(100):
        handler.run('train_all', rollout=30, a_rate=0.001, c_rate=0.005, decay=0.99, render=True)
        print('-',end='')
    print('Completed 100 Training Iterations\n')
    handler.save()

In [ ]:
handler.run_constant_training(10000,render=True, decay=0.98, a_rate=0.001, c_rate=0.01)

In [ ]:
handler.load()

In [ ]:
handler.save()

In [ ]:
handler.play()